# Theoretical Precision of the Portalevel - An Interactive Analysis

What follows is a short analysis of the maximum theoretical precision of the Portalevel. Note that a simplified approach is used based on a number of assumptions and that the practical precision of Portalevel devices is certainly larger than the theoretical values calculated here. Thus, this analysis serves primarily as an understanding of what might be achievable in an idealised setting and **any values stated here must not be quoted as indicative of the genuine precision of any Portalevel devices.**

### An Assumption about the Number of Reflections

Firstly, it is assumed that the measured part of the signal travels exclusively within the walls of the cylinder and that it takes the path of least resistance, which is assumed to be the most direct path around the circumference. Along this path, each "reflection" off of the inner wall is tangent to the circle i.e. it is more of a "brush". Using geometrical principles it is trivial to prove that, along this path, the number of reflections, *n* on the inner wall of the cylinder will be equal to the following expression for a cylinder of outer radius, *r* and wall thickness, *t*.

$$
n = \frac{\pi}{\arccos{\left(\frac{r-t}{r}\right)}}
$$

In [1]:
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from ipywidgets import interact

@interact
def calculate_n(c=(0.25, 5.0, 0.01), wt=(0.1, 40.0, 0.1)):
    #  create table for circumference, c (m) and wall thickness, wt (mm)
    
    t = wt / 1000  # convert to (m)
    r = c / 2 / math.pi  # radius (m)
    theta = 2 * math.acos((r - t) / r)

    n = math.ceil(2 * math.pi / theta)  # number of reflections (rounded up to nearest integer)

    print("There will be {n} reflections for a cylinder of wall thickness {wt}mm and circumference {c}m.".format(wt=wt, c=c,
                                                                                                                 n=n))
    
    fig, ax = plt.subplots()
    circle1 = plt.Circle((0, 0), r, color='black', fill=False)
    circle2 = plt.Circle((0, 0), r - t, color='black', fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
    ax.set_aspect(1)
    ax.set_frame_on(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    for i in range(n):
        x1 = - r * math.cos(i * theta - math.pi / 2)
        y1 = r * math.sin(i * theta - math.pi / 2)
        x2 = - r * math.cos((i + 1) * theta - math.pi / 2)
        y2 = r * math.sin((i + 1) * theta - math.pi / 2)
        ax.plot([x1, x2], [y1, y2], color='black', linewidth=1)  # x axis
    
    

interactive(children=(FloatSlider(value=2.62, description='c', max=5.0, min=0.25, step=0.01), FloatSlider(valu…

### Relative Amplitudes in a Rectangular Crystal

It is the difference in the reflection coefficients above liquid level and below liquid level that is the key physical principle behind the ability of ultrasound to differentiate the two states. Thus, in order to attempt to gain some idea of the maximum precision, we must find the separation either side of the liquid level at which the difference in the signal received is substantial enough to be picked up by the Portalevel sensor. To make sense of this, let us observe the following interactive diagrams concerning the rectangular piezo-electric crystal used by the Portalevel TOUCH.

In [2]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy

h = 20  # height of crystal (mm)
w = 5  # width of crystal (mm)

@interact

def create_crystal_images(L=(0, h / 2, 0.1), c=(0.25, 5.0, 0.01), wt=(0.1, 40.0, 0.1),
                          a=["CO\u2082", "Novec\u2122 1230", "FM-200\u2122"]):

    fig, axs = plt.subplots(2, 2, gridspec_kw={'height_ratios': [2, 1]})
    (ax1, ax2), (ax3, ax4) = axs
    colour = 'blue'
    for ax in [ax1, ax2]:        
        rect1 = Rectangle((w, - h / 2), w, h, linewidth=1, color='black', fill=False)
        rect2 = Rectangle((w, L), w, h / 2 - L, linewidth=1, color='gray')
        rect3 = Rectangle((w, - h / 2), w, h / 2 + L, linewidth=1, color=colour)
        ax.add_patch(rect3)
        ax.add_patch(rect2)
        ax.add_patch(rect1)
        ax.plot([0, 3 * w], [L, L], color='black', linewidth=1, linestyle ='--')
        ax.arrow(w, h / 2 + 1, w, 0, head_width=0.5, head_length=0.25, linewidth=0.5, color='black', length_includes_head=True)
        ax.arrow(2 * w, h / 2 + 1, -w, 0, head_width=0.5, head_length=0.25, linewidth=0.5, color='black', length_includes_head=True)
        ax.text(w * 1.5 - 0.8, h / 2 + 1.7, 'w')
        ax.text(w * 1.5 - 1, h / 4 + L / 2 - 0.4, 'X\u208A', fontsize=15)
        ax.text(w * 1.5 - 1, - h / 4 + L / 2 - 0.4, 'X\u208B', fontsize=15)

        # axes formatting
        ax.set_frame_on(False)
        ax.get_xaxis().set_visible(False)
        ax.plot([0, 3 * w], [0, 0], color='black', linewidth=1)  # x axis
        ax.plot([0, 0], [- h / 2 - 2, h / 2 + 2], color='black', linewidth=1)  # y axis
        ax.set_yticks([-h/2, 0, L, h/2])
        ax.set_xlim([0, h])
        ax.set_ylim([- h / 2 - 2, h / 2 + 2])
        ax.set_aspect(1)
        L=-L
        colour = 'red'
    
    
    ax1.set_yticklabels([r'$-\frac{h}{2}$', 0, r'$L$', r'$\frac{h}{2}$'])
    ax2.set_yticklabels([r'$-\frac{h}{2}$', 0, r'$-L$', r'$\frac{h}{2}$'])
    ax1.set_ylabel('Height (mm)')
    
    if a == "CO\u2082":
        r_g = 0.996751447224324  # Reflection Coefficient (steel-gas)
        r_l = 0.96941482         # Reflection Coefficient (steel-liquid)
    elif a == "Novec\u2122 1230":
        r_g = 0.999706997
        r_l = 0.927788768
    elif a == "FM-200\u2122":
        r_g = 0.999706997
        r_l = 0.94601711
        
    wt /= 1000  # convert to (m)
    r = c / 2 / math.pi  # radius (m)

    n = math.ceil(math.pi / math.acos((r - wt) / r))  # number of reflections (rounded up to nearest integer)
        
    a_plus_hat = (r_g ** n * (h / 2 - L) + r_l ** n * (h / 2 + L)) / (h * r_g ** n)
    a_minus_hat = (r_g ** n * (h / 2 + L) + r_l ** n * (h / 2 - L)) / (h * r_g ** n)
    xs = numpy.linspace(0, math.pi, 1000)
    ys1 = [a_plus_hat * math.sin(x) for x in xs]
    ax3.plot(xs, ys1, color='blue', linewidth=1, label=r'$\hat{A}_L$' )
    ys2 = [a_minus_hat * math.sin(x) for x in xs]
    ax3.plot(xs, ys2, color='red', linewidth=1, label=r'$\hat{A}_{-L}$' )
    ys3 = [ys2[i] - ys1[i] for i in range(len(xs))]
    ax4.plot(xs, ys3, color='green', linewidth=1, label=r'$\Delta\hat{A}_L$' )
       
    for ax in [ax3, ax4]:
        ax.set_frame_on(False)
        ax.get_xaxis().set_visible(False)
        ax.plot([0, 0], [0, 1], color='black', linewidth=1)  # y axis
        ax.set_yticks([0, 0.5, 1])
        ax.set_yticklabels(['0%', '50%', '100%'])
        ax.set_xlim([0, math.pi])
        ax.set_ylim([0, 1])
        
    ax3.set_ylabel('Signal Amplitude (%)')

    handles, labels = [(a + b) for a, b in zip(ax3.get_legend_handles_labels(), ax4.get_legend_handles_labels())]
    fig.legend(handles, labels, loc="upper right")

    wt = int(wt * 100000) / 1000
    print("h = height of crystal ({} mm)".format(h))
    print("w = width of crystal ({} mm)".format(w))
    print("L = liquid level above centre line ({} mm)".format(L))
    print("c = circumference of cylinder ({} m)".format(c))
    print("wt = wall thickness of cylinder ({} mm)".format(wt))
    print("a = agent ({})".format(a))
    plt.show()


interactive(children=(FloatSlider(value=5.0, description='L', max=10.0), FloatSlider(value=2.62, description='…

Observing first the diagram in the top left corner of the above figure, the area of the crystal that is exposed to the gas-steel interface, we call $X_{+}$ (coloured grey) and the area that is exposed to the liquid-steel interface we call $X_{-}$ (coloured, in this case, blue). Therefore, if we assume that the signal is received homogeneously over the entirety of the crystal face, the total amplitude received is proportional to $A_{L}={R_{g}}^{n}X_{+}+{R_{l}}^{n}X_{-}$ where $R_{g}$ is the coefficient of reflection at the gas-steel interface, $R_{l}$ is the coefficient of reflection at the gas-liquid interface and $n$ is the number of reflections that take place (see earlier discussion).

The subscript $L$ indicates that this amplitude may be calculated for any liquid level that we choose. Now we shall look at the same scenario, but with the liquid level positioned an equal distance **beneath** the centre line of the crystal. This is the diagram in the top right corner of the above figure.

Let us call the amplitude for the second diagram '$A_{-L}$' such that we can express the difference of the amplitudes as $A_{-L}-A_{L}$. The difference in amplitude between the two positions, normalised by the maximum amplitude (which occurs when the entire crystal is positioned above the liquid level) is therefore given by $$\Delta\hat{A}_L = \frac{A_{-L}-A_{L}}{A_{-\frac{h}{2}}}=\frac{\hat{A}_{-L}-\hat{A}_{L}}{{R_g}^n}$$

In the case of a rectangular crystal, we have
\begin{align}
A_{L}&={R_{g}}^{n}w\left(\frac{h}{2}-L\right)+{R_{l}}^{n}w\left(\frac{h}{2}+L\right)\\
A_{-L}&={R_{g}}^{n}w\left(\frac{h}{2}+L\right)+{R_{l}}^{n}w\left(\frac{h}{2}-L\right)\\
A_{-\frac{h}{2}}&={R_{g}}^{n}wh
\end{align}.
Therefore,
$$\Delta\hat{A}_L=\frac{2L}{h}\frac{{R_g}^n-{R_l}^n}{{R_g}^n}$$.

Now that we know the expression for the normalised amplitude difference between the two crystal positions, we can equate this to a property of the Portalevel that determines the signal resolution and solve for L to find the precision. The property we equate to is, in fact, the inverse of the Signal-to-Noise Ratio.

$$\frac{1}{SNR}=\Delta\hat{A}_L$$.

Thus, in the above case, we find that

$$L=\frac{h}{2\cdot SNR}\frac{{R_g}^n}{{R_g}^n-{R_l}^n}$$

where L is the precision in millimetres.

### Relative Amplitudes in a Circular Crystal

Before we use the above equation to perform some calculations, we will now turn to the more tricky example of the circular crystal used by the Portalevel MAX and all preceding Portalevels. Using the fact that the area of a segment of a circle of central angle $\theta$ and radius $r$ is given by

$$X=\frac{\theta-\sin\theta}{2}r^2$$,

and using simple trigonometry, we find that the area of the crystal above a liquid level, L is given by the expression

$$X_+=\left[\arccos\left(\frac{L}{r}\right)-\frac{L}{r}\sin\left(\arccos\left(\frac{L}{r}\right)\right)\right]r^2$$

Taking the difference between $X_+$ and $X_-$ and normalising by the maximum value, (and after a bit of algebraic simplification) we find that the corresponding normalised amplitude difference may be expressed as

$$\Delta\hat{A}_L=\frac{{R_g}^n-{R_l}^n}{\pi{R_g}^n}\left[\pi-2\arccos\left(\frac{2L}{h}\right)+\frac{4L}{h}\sqrt{1-\frac{4L^2}{h^2}}\right]$$

Equating to the inverse of the Signal-to-Noise Ratio, we find that solving for $L$ is nontrivial and thus the solution will be computed numerically.

In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy

d = 15  # diameter of crystal (mm)

@interact

def create_crystal_images(L=(0, d / 2, 0.1), c=(0.25, 5.0, 0.01), wt=(0.1, 40.0, 0.1),
                          a=["CO\u2082", "Novec\u2122 1230", "FM-200\u2122"]):

    fig, axs = plt.subplots(2, 2, gridspec_kw={'height_ratios': [2, 1]})
    (ax1, ax2), (ax3, ax4) = axs
    colour1 = 'blue'
    colour2 = 'gray'
    for ax in [ax1, ax2]:
        centre_x = d / 2 + 1
        circle = Circle((centre_x, 0), d / 2, linewidth=1, color='black', fill=False)
        circle_fill = Circle((centre_x, 0), d / 2, linewidth=1, color=colour1)
        x = numpy.linspace(centre_x - math.sqrt((d / 2) ** 2 - L ** 2), centre_x + math.sqrt((d / 2) ** 2 - L ** 2), 100)
        y1 = math.copysign(1, L) * numpy.sqrt((d / 2) ** 2 - (x - centre_x) ** 2)
        y2 = [L] * 100
        ax.add_patch(circle_fill)
        ax.fill_between(x, y2, y1, color=colour2, zorder=2)
        ax.add_patch(circle)
        ax.plot([0, 2 * d], [L, L], color='black', linewidth=1, linestyle ='--')
        ax.text(d / 2, d / 4 + L / 2 - 1, 'X\u208A', fontsize=15)
        ax.text(d / 2, - d / 4 + L / 2 - 1, 'X\u208B', fontsize=15)

        # axes formatting
        ax.set_frame_on(False)
        ax.get_xaxis().set_visible(False)
        ax.plot([0, d + 4], [0, 0], color='black', linewidth=1)  # x axis
        ax.plot([0, 0], [- d / 2 - 2, d / 2 + 2], color='black', linewidth=1)  # y axis
        ax.set_yticks([-d/2, 0, L, d/2])
        ax.set_xlim([0, d + 4])
        ax.set_ylim([- d / 2 - 2, d / 2 + 2])
        ax.set_aspect(1)
        
        # These are the only things that differ for ax2
        L=-L
        colour1 = 'gray'
        colour2 = 'red'
        
    L = abs(L)  # set L back to what it should be for later
    
    
    ax1.set_yticklabels([r'$-\frac{d}{2}$', 0, r'$L$', r'$\frac{d}{2}$'])
    ax2.set_yticklabels([r'$-\frac{d}{2}$', 0, r'$-L$', r'$\frac{d}{2}$'])
    ax1.set_ylabel('Height (mm)')
    
    if a == "CO\u2082":
        r_g = 0.996751447224324  # Reflection Coefficient (steel-gas)
        r_l = 0.96941482         # Reflection Coefficient (steel-liquid)
    elif a == "Novec\u2122 1230":
        r_g = 0.999706997
        r_l = 0.927788768
    elif a == "FM-200\u2122":
        r_g = 0.999706997
        r_l = 0.94601711
        
    wt /= 1000  # convert to (m)
    r = c / 2 / math.pi  # radius (m)

    n = math.ceil(math.pi / math.acos((r - wt) / r))  # number of reflections (rounded up to nearest integer)
    
    x_plus_hat = (math.acos(2 * L / d) - 2 * L / d * math.sqrt(1 - (2 * L / d) ** 2)) / math.pi
    x_minus_hat = 1 - x_plus_hat
    a_plus_hat = (r_g ** n * x_plus_hat + r_l ** n * x_minus_hat) / r_g ** n
    a_minus_hat = (r_g ** n * x_minus_hat + r_l ** n * x_plus_hat) / r_g ** n
    xs = numpy.linspace(0, math.pi, 1000)
    ys1 = [a_plus_hat * math.sin(x) for x in xs]
    ax3.plot(xs, ys1, color='blue', linewidth=1, label=r'$\hat{A}_L$' )
    ys2 = [a_minus_hat * math.sin(x) for x in xs]
    ax3.plot(xs, ys2, color='red', linewidth=1, label=r'$\hat{A}_{-L}$' )
    ys3 = [ys2[i] - ys1[i] for i in range(len(xs))]
    ax4.plot(xs, ys3, color='green', linewidth=1, label=r'$\Delta\hat{A}_L$' )
       
    for ax in [ax3, ax4]:
        ax.set_frame_on(False)
        ax.get_xaxis().set_visible(False)
        ax.plot([0, 0], [0, 1], color='black', linewidth=1)  # y axis
        ax.set_yticks([0, 0.5, 1])
        ax.set_yticklabels(['0%', '50%', '100%'])
        ax.set_xlim([0, math.pi])
        ax.set_ylim([0, 1])
        
    ax3.set_ylabel('Signal Amplitude (%)')

    handles, labels = [(a + b) for a, b in zip(ax3.get_legend_handles_labels(), ax4.get_legend_handles_labels())]
    fig.legend(handles, labels, loc="upper right")

    wt = int(wt * 100000) / 1000
    print("d = diameter of crystal ({} mm)".format(h))
    print("L = liquid level above centre line ({} mm)".format(L))
    print("c = circumference of cylinder ({} m)".format(c))
    print("wt = wall thickness of cylinder ({} mm)".format(wt))
    print("a = agent ({})".format(a))
    plt.show()

interactive(children=(FloatSlider(value=3.7, description='L', max=7.5), FloatSlider(value=2.62, description='c…

### Table of Precisions

Now that we have determined a way of calculating the precision for both types of crystal, we may observe the interactive table below. The SNR values are dependent on the electronics of the device and have been estimated by observing the waveforms of the Portalevel MAX and the Portalevel TOUCH. For the circular crystal we use an SNR of 100 and for the rectangular crystal, an SNR of 133.

In [4]:
from IPython.display import HTML, display
import tabulate
from scipy.optimize import newton_krylov

def circ_rhs(p, r_g, r_l, ch, n, snr):
    #  Given the precision, calculate the normalised amplitude difference for a circular crystal.
    #  Use this with Newton-Krylov to solve backwards for the precision
    #  p: precision (mm), float
    #  r_g: steel-gas reflection coefficient, float
    #  r_l: steel-liquid reflection coefficient, float
    #  ch: crystal height (mm), float
    #  n: number of reflections, int
    #  snr: signal-to-noise ratio, float
    #
    #  returns zero
    
    delta_a_hat = (r_g ** n - r_l ** n) / math.pi / r_g ** n * (math.pi - 2 * math.acos(2 * p / ch)
                                                                + 4 * p / ch * math.sqrt(1 - 4 * p ** 2 / ch ** 2))
        
    return delta_a_hat - 1 / snr
    

def find_precision(a, c, wt, cs, ch, snr):
    #  a: agent in cylinder, str
    #  c: circumference of cylinder (m), float
    #  wt: wall thickness of cylinder (mm), float
    #  cs: crystal shape, str
    #  ch: crystal height, str
    #  snr: signal-to-noise ratio, float
    #
    #  returns p: precision (mm), float
    
    if a == 'CO2':
        r_g = 0.996751447224324  # Reflection Coefficient (steel-gas)
        r_l = 0.96941482         # Reflection Coefficient (steel-liquid)
    elif a == 'Novec':
        r_g = 0.999706997
        r_l = 0.927788768
    elif a == 'FM200':
        r_g = 0.999706997
        r_l = 0.94601711
        
    wt /= 1000  # convert to metres
    r = c / 2 / math.pi  # radius (m)
    n = math.ceil(math.pi / math.acos((r - wt) / r))  # number of reflections (rounded up to nearest integer)
    
    if cs == 'rectangular':
        p = ch * r_g ** n / (2 * snr * (r_g ** n - r_l ** n))
    elif cs == 'circular':
        p = newton_krylov(lambda p:circ_rhs(p, r_g, r_l, ch, n, snr), 0.5)
    
    p_string = '\u00B1{:.4f}mm'.format(p)
    return p_string

@interact
def create_table(c=(0.25, 5.0, 0.01), wt=(0.1, 40.0, 0.1)):
    #  create table for circumference, c (m) and wall thickness, wt (mm)
    print("c: circumference of cylinder ({}m)".format(c))
    wt = int(wt * 10) / 10
    print("wt: wall thickness of cylinder ({}mm)".format(wt))
    
    table = [["", "CO\u2082", "Novec\u2122 1230", "FM-200\u2122"],
             ["Rectangular Crystal", find_precision('CO2', c, wt, 'rectangular', 20, 133),
              find_precision('Novec', c, wt, 'rectangular', 20, 133), find_precision('FM200', c, wt, 'rectangular', 20, 133)],
             ["Circular Crystal", find_precision('CO2', c, wt, 'circular', 15, 100),
              find_precision('Novec', c, wt, 'circular', 15, 100), find_precision('FM200', c, wt, 'circular', 15, 100)]]
    
    display(HTML(tabulate.tabulate(table, tablefmt='html')))
  


interactive(children=(FloatSlider(value=2.62, description='c', max=5.0, min=0.25, step=0.01), FloatSlider(valu…